# Exercices Supplémentaires
### Name : Bouguedra Mohammed Yazid / G4 / S2
## Exercice 3
### Question 1 : 

In [33]:
from numpy.random import uniform
import numpy as np
import simpy

moy_clients = 12
moy_services = 30
x=1/moy_clients
y=1/moy_services
file = []
etat = 0
arrive = []
fin = []
enAttente = []

#Monkey patching is a sublte art, I started wrapping the simpy.resource call to try and extract data but i didn't use it after all...
class MonitoredResource(simpy.Resource):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.data = []

    def request(self, *args, **kwargs):
        self.data.append((self._env.now, len(self.queue),self.count))
        return super().request(*args, **kwargs)

    def release(self, *args, **kwargs):
        self.data.append((self._env.now, len(self.queue),self.count))
        return super().release(*args, **kwargs)
    
    
def arrive_client():
    return uniform() <=x

def service_client():
    return uniform() <=y

def traitement_client(env, num_client, serveur):
    global file, etat
    
    file = file+[num_client]
    
    etat = etat + 1
    print(env.now,': client', num_client,'arrive, clients en attente : ', file)
    
    
    with serveur.request() as req:
        yield req
        print(env.now,': debut du traitement du client', num_client)
        temps = 0
        while True : 
            temps = temps+1
            if(service_client()):
                yield env.timeout(temps)
                print(env.now,': fin du traitement du client ', num_client)
                fin[num_client] = env.now
                file.remove(num_client)
                etat = etat -1
                break

def generate_clients(env,serveur):
    global arrive, etat
    num_client=0
    temps=0
    
    while True:
        temps = temps +1
        if(arrive_client()):
            yield env.timeout(temps)
            enAttente.append(len(file))
            arrive.append(env.now)
            fin.append(env.now)
            env.process(traitement_client(env, num_client, serveur))
            num_client+=1
            temps=0
    
env = simpy.Environment()
serveur = MonitoredResource(env, capacity=2)
env.process(generate_clients(env, serveur))

print('Début de la simulation')
env.run(until=8*60)
print('Fin de a simulation')
attente = np.array(fin)-np.array(arrive)
enAttente = np.array(enAttente)

print('\nAttente moyenne : ', attente.mean(),'Attente max : ', attente.max(), 'Attente min : ' , attente.min()  )
print('\nNomnre de clients en attente moyen : ', enAttente.mean(),'Nomnre de clients en attente max : ', enAttente.max()  )


Début de la simulation
13 : client 0 arrive, clients en attente :  [0]
13 : debut du traitement du client 0
16 : fin du traitement du client  0
37 : client 1 arrive, clients en attente :  [1]
37 : debut du traitement du client 1
39 : client 2 arrive, clients en attente :  [1, 2]
39 : debut du traitement du client 2
45 : fin du traitement du client  1
46 : client 3 arrive, clients en attente :  [2, 3]
46 : debut du traitement du client 3
52 : fin du traitement du client  3
59 : client 4 arrive, clients en attente :  [2, 4]
59 : debut du traitement du client 4
64 : client 5 arrive, clients en attente :  [2, 4, 5]
68 : fin du traitement du client  4
68 : debut du traitement du client 5
85 : client 6 arrive, clients en attente :  [2, 5, 6]
96 : client 7 arrive, clients en attente :  [2, 5, 6, 7]
99 : client 8 arrive, clients en attente :  [2, 5, 6, 7, 8]
107 : client 9 arrive, clients en attente :  [2, 5, 6, 7, 8, 9]
108 : client 10 arrive, clients en attente :  [2, 5, 6, 7, 8, 9, 10]
112 


#### La moyenne des clients en attente est de ~7-8 (Du moins, c'est le cas des jorunées avec 35-40 clients) Il faut donc prévoir aumoins 5 chaises de plus des 2.

### -----------
### Question 2 :

In [5]:
from numpy.random import uniform
import numpy as np
import simpy

moy_clients = 12
moy_services = 30
x=1/moy_clients
y=1/moy_services
file = []
etat = 0
arrive = []
fin = []
enAttente = []
rejet = 0
#Monkey patching is a sublte art, I started wrapping the simpy.resource call to try and extract data but i didn't use it after all...
class MonitoredResource(simpy.Resource):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.data = []

    def request(self, *args, **kwargs):
        self.data.append((self._env.now, len(self.queue),self.count))
        return super().request(*args, **kwargs)

    def release(self, *args, **kwargs):
        self.data.append((self._env.now, len(self.queue),self.count))
        return super().release(*args, **kwargs)
    
    
def arrive_client():
    return uniform() <=x

def service_client():
    return uniform() <=y

def traitement_client(env, num_client, serveur):
    global file, etat, rejet
    
    if(len(file) >= 7):
        rejet += 1
        return
    
    file = file+[num_client]
    
    etat = etat + 1
    print(env.now,': client', num_client,'arrive, clients en attente : ', len(file))
    
    
    with serveur.request() as req:
        yield req
        #print(env.now,': debut du traitement du client', num_client)
        temps = 0
        while True : 
            temps = temps+1
            if(service_client()):
                yield env.timeout(temps)
                #print(env.now,': fin du traitement du client ', num_client)
                fin[num_client] = env.now
                file.remove(num_client)
                etat = etat -1
                break

def generate_clients(env,serveur):
    global arrive, etat
    num_client=0
    temps=0
    
    while True:
        temps = temps +1
        if(arrive_client()):
            yield env.timeout(temps)
            enAttente.append(len(file))
            arrive.append(env.now)
            fin.append(env.now)
            env.process(traitement_client(env, num_client, serveur))
            num_client+=1
            temps=0
            
    
env = simpy.Environment()
serveur = MonitoredResource(env, capacity=2)
env.process(generate_clients(env, serveur))

print('Début de la simulation')
env.run(until=8*60)
print('Fin de a simulation')
attente = np.array(fin)-np.array(arrive)
enAttente = np.array(enAttente)

print('\nAttente moyenne : ', attente.mean(),'Attente max : ', attente.max(), 'Attente min : ' , attente.min()  )
print('\nNomnre de clients en attente moyen : ', enAttente.mean(),'Nomnre de clients en attente max : ', enAttente.max())
print('\nRejets : ', rejet)


Début de la simulation
47 : client 0 arrive, clients en attente :  1
57 : client 1 arrive, clients en attente :  2
58 : client 2 arrive, clients en attente :  3
65 : client 3 arrive, clients en attente :  3
67 : client 4 arrive, clients en attente :  3
73 : client 5 arrive, clients en attente :  4
93 : client 6 arrive, clients en attente :  5
108 : client 7 arrive, clients en attente :  5
126 : client 8 arrive, clients en attente :  4
132 : client 9 arrive, clients en attente :  5
133 : client 10 arrive, clients en attente :  6
138 : client 11 arrive, clients en attente :  7
156 : client 14 arrive, clients en attente :  6
160 : client 15 arrive, clients en attente :  7
180 : client 16 arrive, clients en attente :  7
202 : client 17 arrive, clients en attente :  6
203 : client 18 arrive, clients en attente :  7
222 : client 20 arrive, clients en attente :  7
241 : client 21 arrive, clients en attente :  7
269 : client 22 arrive, clients en attente :  4
270 : client 23 arrive, clients en

#### Environs 10-15 Rejets par jour

#### -----------
### Question 3 

In [3]:
from numpy.random import uniform
import numpy as np
import simpy

moy_clients = 12
moy_services = 30
x=1/moy_clients
y=1/moy_services
file = []
etat = 0
arrive = []
fin = []
enAttente = []
rejet = 0
caisse=0
#Monkey patching is a sublte art, I started wrapping the simpy.resource call to try and extract data but i didn't use it after all...
class MonitoredResource(simpy.Resource):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.data = []

    def request(self, *args, **kwargs):
        self.data.append((self._env.now, len(self.queue),self.count))
        return super().request(*args, **kwargs)

    def release(self, *args, **kwargs):
        self.data.append((self._env.now, len(self.queue),self.count))
        return super().release(*args, **kwargs)
    
    
def arrive_client():
    return uniform() <=x

def service_client():
    return uniform() <=y

def traitement_client(env, num_client, serveur):
    global file, etat, rejet, caisse
    
    if(len(file) >= 7):
        rejet += 1
        return
    
    file = file+[num_client]
    
    etat = etat + 1
    print(env.now,': client', num_client,'arrive, clients en attente : ', len(file))
    
    
    with serveur.request() as req:
        yield req
        #print(env.now,': debut du traitement du client', num_client)
        temps = 0
        while True : 
            temps = temps+1
            if(service_client()):
                yield env.timeout(temps)
                #print(env.now,': fin du traitement du client ', num_client)
                fin[num_client] = env.now
                file.remove(num_client)
                etat = etat -1
                caisse+=400
                break

def generate_clients(env,serveur):
    global arrive, etat
    num_client=0
    temps=0
    
    while True:
        temps = temps +1
        if(arrive_client()):
            yield env.timeout(temps)
            enAttente.append(len(file))
            arrive.append(env.now)
            fin.append(env.now)
            env.process(traitement_client(env, num_client, serveur))
            num_client+=1
            temps=0
            
    
env = simpy.Environment()
serveur = MonitoredResource(env, capacity=3)
env.process(generate_clients(env, serveur))

print('Début de la simulation')
env.run(until=30*8*60)
print('Fin de a simulation')
attente = np.array(fin)-np.array(arrive)
enAttente = np.array(enAttente)

print('\nAttente moyenne : ', attente.mean(),'Attente max : ', attente.max(), 'Attente min : ' , attente.min()  )
print('\nNomnre de clients en attente moyen : ', enAttente.mean(),'Nomnre de clients en attente max : ', enAttente.max())
print('\nRejets : ', rejet, "Benefice net : ", caisse-270000)


Début de la simulation
30 : client 0 arrive, clients en attente :  1
45 : client 1 arrive, clients en attente :  2
65 : client 2 arrive, clients en attente :  1
87 : client 3 arrive, clients en attente :  1
96 : client 4 arrive, clients en attente :  2
134 : client 5 arrive, clients en attente :  3
141 : client 6 arrive, clients en attente :  4
154 : client 7 arrive, clients en attente :  5
184 : client 8 arrive, clients en attente :  1
186 : client 9 arrive, clients en attente :  2
246 : client 10 arrive, clients en attente :  1
248 : client 11 arrive, clients en attente :  2
260 : client 12 arrive, clients en attente :  3
275 : client 13 arrive, clients en attente :  2
292 : client 14 arrive, clients en attente :  3
294 : client 15 arrive, clients en attente :  4
303 : client 16 arrive, clients en attente :  3
308 : client 17 arrive, clients en attente :  2
312 : client 18 arrive, clients en attente :  3
320 : client 19 arrive, clients en attente :  4
325 : client 20 arrive, clients 

3422 : client 297 arrive, clients en attente :  5
3437 : client 298 arrive, clients en attente :  5
3484 : client 299 arrive, clients en attente :  2
3499 : client 300 arrive, clients en attente :  3
3504 : client 301 arrive, clients en attente :  4
3510 : client 302 arrive, clients en attente :  5
3513 : client 303 arrive, clients en attente :  6
3548 : client 304 arrive, clients en attente :  2
3549 : client 305 arrive, clients en attente :  3
3555 : client 306 arrive, clients en attente :  2
3557 : client 307 arrive, clients en attente :  3
3559 : client 308 arrive, clients en attente :  4
3576 : client 309 arrive, clients en attente :  4
3584 : client 310 arrive, clients en attente :  5
3604 : client 311 arrive, clients en attente :  4
3649 : client 312 arrive, clients en attente :  1
3677 : client 313 arrive, clients en attente :  1
3682 : client 314 arrive, clients en attente :  2
3686 : client 315 arrive, clients en attente :  3
3690 : client 316 arrive, clients en attente :  4


5910 : client 503 arrive, clients en attente :  6
5912 : client 504 arrive, clients en attente :  7
5915 : client 505 arrive, clients en attente :  7
5938 : client 506 arrive, clients en attente :  6
5953 : client 507 arrive, clients en attente :  5
5986 : client 508 arrive, clients en attente :  4
5989 : client 509 arrive, clients en attente :  5
6005 : client 510 arrive, clients en attente :  6
6045 : client 511 arrive, clients en attente :  6
6050 : client 512 arrive, clients en attente :  7
6076 : client 516 arrive, clients en attente :  7
6082 : client 518 arrive, clients en attente :  7
6098 : client 519 arrive, clients en attente :  6
6139 : client 520 arrive, clients en attente :  5
6146 : client 521 arrive, clients en attente :  6
6157 : client 522 arrive, clients en attente :  6
6166 : client 523 arrive, clients en attente :  6
6171 : client 524 arrive, clients en attente :  7
6239 : client 528 arrive, clients en attente :  7
6300 : client 531 arrive, clients en attente :  3


8768 : client 734 arrive, clients en attente :  5
8770 : client 735 arrive, clients en attente :  6
8787 : client 736 arrive, clients en attente :  5
8811 : client 737 arrive, clients en attente :  3
8822 : client 738 arrive, clients en attente :  2
8825 : client 739 arrive, clients en attente :  3
8839 : client 740 arrive, clients en attente :  3
8849 : client 741 arrive, clients en attente :  3
8853 : client 742 arrive, clients en attente :  4
8879 : client 743 arrive, clients en attente :  4
8895 : client 744 arrive, clients en attente :  2
8900 : client 745 arrive, clients en attente :  2
8912 : client 746 arrive, clients en attente :  2
8917 : client 747 arrive, clients en attente :  2
8958 : client 748 arrive, clients en attente :  2
8966 : client 749 arrive, clients en attente :  3
8969 : client 750 arrive, clients en attente :  4
8997 : client 751 arrive, clients en attente :  2
9000 : client 752 arrive, clients en attente :  3
9009 : client 753 arrive, clients en attente :  4


11296 : client 958 arrive, clients en attente :  3
11297 : client 959 arrive, clients en attente :  4
11303 : client 960 arrive, clients en attente :  4
11315 : client 961 arrive, clients en attente :  2
11329 : client 962 arrive, clients en attente :  3
11340 : client 963 arrive, clients en attente :  3
11344 : client 964 arrive, clients en attente :  3
11353 : client 965 arrive, clients en attente :  4
11354 : client 966 arrive, clients en attente :  5
11362 : client 967 arrive, clients en attente :  5
11372 : client 968 arrive, clients en attente :  6
11378 : client 969 arrive, clients en attente :  7
11400 : client 972 arrive, clients en attente :  7
11407 : client 973 arrive, clients en attente :  7
11431 : client 974 arrive, clients en attente :  7
11439 : client 976 arrive, clients en attente :  7
11466 : client 979 arrive, clients en attente :  6
11468 : client 980 arrive, clients en attente :  7
11480 : client 982 arrive, clients en attente :  7
11492 : client 984 arrive, clie

14030 : client 1177 arrive, clients en attente :  2
14037 : client 1178 arrive, clients en attente :  2
14043 : client 1179 arrive, clients en attente :  3
14046 : client 1180 arrive, clients en attente :  4
14068 : client 1181 arrive, clients en attente :  4
14071 : client 1182 arrive, clients en attente :  5
14074 : client 1183 arrive, clients en attente :  6
14076 : client 1184 arrive, clients en attente :  7
14108 : client 1187 arrive, clients en attente :  6
14115 : client 1188 arrive, clients en attente :  6
14137 : client 1189 arrive, clients en attente :  6
14138 : client 1190 arrive, clients en attente :  7
14142 : client 1191 arrive, clients en attente :  7
14161 : client 1192 arrive, clients en attente :  7
14184 : client 1193 arrive, clients en attente :  6
14194 : client 1194 arrive, clients en attente :  5
14242 : client 1195 arrive, clients en attente :  2
14246 : client 1196 arrive, clients en attente :  3
14269 : client 1197 arrive, clients en attente :  2
14318 : clie

#### Benefice net (moins le salaire des coiffeurs) apres plusieurs tests de 30 jours :  
#### 2 coiffeurs : 17 milions en moyenne
#### 3 coiffeurs : ~17.5 milions en moyenne
#### Critères de performance : Beaucoup moins de rejets, temps d'attente diminué mais le benefice n'est pas tès interessant je dirais que c'est pas forcémment une bonne idée d'embaucher un 3eme coiffeur.

## Exercice 4


In [14]:
from random import randint
from numpy.random import uniform
import numpy as np
import simpy


#I know, this code could've been simplified in so many ways... but hey ! it works !

carburant = ["Gasoil","Gasoil","Gasoil","Sans plomb","Sans plomb","Super","Super","Super","Super","Super"]
file_gasoil = []
file_sansplomb = []
file_super = []

enAttente_gasoil = []
enAttente_sansplomb = []
enAttente_super = []

etat_gasoil = 0
etat_sansplomb = 0
etat_super = 0

rejet_gasoil = 0
rejet_sansplomb = 0
rejet_super = 0

reservoir_gasoil = 15000
reservoir_sansplomb = 15000
reservoir_super = 30000

super_count = 0
def arrive_voiture():
    if(uniform()<= 5/6):
        return carburant[randint(0,9)]
    return False

def service_voiture():
    return uniform() <= 1/10 #1 voiture toutes les 5 minutes --> 1 voiture toutes les  5*2 30 secondes

def generate_cars(env,g,sp,s):
    global arrive, etat
    num_car=0
    temps=0
    
    while True:
        temps = temps +1 #1 temps = 30 sec
        voiture = arrive_voiture()
        if(voiture):
            yield env.timeout(temps)
            if voiture == "Gasoil" : env.process(traitement_voiture_gasoil(env, num_car, g))
            elif voiture == "Sans plomb" : env.process(traitement_voiture_sansplomb(env, num_car, sp))
            elif voiture == "Super" : env.process(traitement_voiture_super(env, num_car, s))
            num_car+=1
            temps=0

def traitement_voiture_gasoil(env, num_car, serveur):
    global file_gasoil, etat_gasoil, rejet_gasoil, reservoir_gasoil
    
    enAttente_gasoil.append(len(file_gasoil))
    
    if(len(file_gasoil) >= 4):
        print("[GASOIL] | ", env.now/2,': Voiture ', num_car,' -REJET-')
        rejet_gasoil += 1
        return
    
    file_gasoil = file_gasoil+[num_car]
    
    etat_gasoil = etat_gasoil + 1
    
    print("[GASOIL] | [Minute : ", env.now/2 ,'] Voiture ', num_car,'arrive, voitures en attente : ', len(file_gasoil))
    
    
    with serveur.request() as req:
        yield req
        #print("[GASOIL] | ", env.now,': debut du traitement de la voiture ', num_car)
        temps = 0
        while True : 
            temps = temps+1
            if(service_voiture()):
                yield env.timeout(temps)
                #print("[GASOIL] | ", env.now,': fin du traitement de la voiture ', num_car)
                
                reservoir_gasoil -= randint(10,30)
                
                file_gasoil.remove(num_car)
                etat_gasoil = etat_gasoil -1
                break

def traitement_voiture_sansplomb(env, num_car, serveur):
    global file_sansplomb, etat_sansplomb, rejet_sansplomb, reservoir_sansplomb
    
    enAttente_sansplomb.append(len(file_sansplomb))
    
    if(len(file_sansplomb) >= 4):
        print("[SANSPLOMB] | ", env.now/2,': Voiture ', num_car,' -REJET-')
        rejet_sansplomb += 1
        return
    
    file_sansplomb = file_sansplomb+[num_car]
    
    etat_sansplomb = etat_sansplomb + 1
    
    print("[SANSPLOMB] | [Minute : ", env.now/2 ,']: Voiture ', num_car,'arrive, voitures en attente : ', len(file_sansplomb))
    
    
    with serveur.request() as req:
        yield req
        #print("[SANS PLOMB] | ", env.now,': debut du traitement de la voiture ', num_car)
        temps = 0
        while True : 
            temps = temps+1
            if(service_voiture()):
                yield env.timeout(temps)
                #print("[SANS PLOMB] | ", env.now,': fin du traitement de la voiture ', num_car)
                
                reservoir_sansplomb -= randint(10,30)
                
                file_sansplomb.remove(num_car)
                etat_sansplomb = etat_sansplomb -1
                break

def traitement_voiture_super(env, num_car, serveur):
    global file_super, etat_super, rejet_super, super_count, reservoir_super
    
    super_count+=1
    enAttente_super.append(len(file_super))
    
    if(len(file_super) >= 16): # File : 4 pompes * 4 places
        rejet_super += 1
        print("[SUPER] | ", env.now/2,': Voiture ', num_car,' -REJET-')
        return
    
    file_super = file_super+[num_car]
    
    etat_super = etat_super + 1
    
    print("[SUPER] | [Minute : ", env.now/2 ,']: Voiture ', num_car,'arrive, voitures en attente : ', len(file_super))
    
    
    with serveur.request() as req:
        yield req
        #print("[SUPER] | ", env.now,': debut du traitement de la voiture ', num_car)
        temps = 0
        while True : 
            temps = temps+1
            if(service_voiture()):
                yield env.timeout(temps)
                #print("[SUPER] | ", env.now,': fin du traitement de la voiture ', num_car)
                
                reservoir_super -= randint(10,30)
                
                file_super.remove(num_car)
                etat_super = etat_super -1
                break


env = simpy.Environment()
pompe_gasoil = simpy.Resource(env, capacity=1)
pompe_sansplomb = simpy.Resource(env, capacity=1)
pompe_super = simpy.Resource(env, capacity=4)
env.process(generate_cars(env, pompe_gasoil, pompe_sansplomb, pompe_super))

print('Début de la simulation')
env.run(until=24*60*2) # 1 mesure de temps = 30 sec
print('Fin de a simulation' , super_count)

enAttente_gasoil = np.array(enAttente_gasoil)
enAttente_sansplomb = np.array(enAttente_sansplomb)
enAttente_super = np.array(enAttente_super)

print('\n[GASOIL] Nomnre de voitures en attente moyen : ', enAttente_gasoil.mean(),'Nomnre de clients en attente max : ', enAttente_gasoil.max())
print('\n[GASOIL] Rejets : ', rejet_gasoil, "Etat Reservoir : ", reservoir_gasoil, "/150000")
print('\n[SANSPLOMB] Nomnre de voitures en attente moyen : ', enAttente_sansplomb.mean(),'Nomnre de clients en attente max : ', enAttente_sansplomb.max())
print('\n[SANSPLOMB] Rejets : ', rejet_sansplomb, "Etat Reservoir : ", reservoir_sansplomb, "/150000")
print('\n[SUPER] Nomnre de voitures en attente moyen : ', enAttente_super.mean(),'Nomnre de clients en attente max : ', enAttente_super.max())
print('\n[SUPER] Rejets : ', rejet_super, "Etat Reservoir : ", reservoir_super , "/30000")

Début de la simulation
[GASOIL] | [Minute :  0.5 ] Voiture  0 arrive, voitures en attente :  1
[SUPER] | [Minute :  1.0 ]: Voiture  1 arrive, voitures en attente :  1
[SUPER] | [Minute :  1.5 ]: Voiture  2 arrive, voitures en attente :  2
[GASOIL] | [Minute :  2.5 ] Voiture  3 arrive, voitures en attente :  1
[SUPER] | [Minute :  3.0 ]: Voiture  4 arrive, voitures en attente :  2
[SANSPLOMB] | [Minute :  3.5 ]: Voiture  5 arrive, voitures en attente :  1
[SUPER] | [Minute :  4.0 ]: Voiture  6 arrive, voitures en attente :  3
[GASOIL] | [Minute :  4.5 ] Voiture  7 arrive, voitures en attente :  2
[GASOIL] | [Minute :  5.0 ] Voiture  8 arrive, voitures en attente :  3
[SANSPLOMB] | [Minute :  5.5 ]: Voiture  9 arrive, voitures en attente :  2
[SANSPLOMB] | [Minute :  6.0 ]: Voiture  10 arrive, voitures en attente :  3
[SUPER] | [Minute :  6.5 ]: Voiture  11 arrive, voitures en attente :  3
[SUPER] | [Minute :  7.0 ]: Voiture  12 arrive, voitures en attente :  4
[SUPER] | [Minute :  8.0 ]

[SANSPLOMB] | [Minute :  158.0 ]: Voiture  270 arrive, voitures en attente :  4
[GASOIL] |  159.0 : Voiture  271  -REJET-
[SUPER] | [Minute :  159.5 ]: Voiture  272 arrive, voitures en attente :  5
[SUPER] | [Minute :  160.0 ]: Voiture  273 arrive, voitures en attente :  4
[SUPER] | [Minute :  160.5 ]: Voiture  274 arrive, voitures en attente :  4
[GASOIL] |  161.0 : Voiture  275  -REJET-
[GASOIL] |  161.5 : Voiture  276  -REJET-
[GASOIL] |  162.5 : Voiture  277  -REJET-
[SANSPLOMB] |  163.0 : Voiture  278  -REJET-
[SUPER] | [Minute :  163.5 ]: Voiture  279 arrive, voitures en attente :  3
[GASOIL] |  164.0 : Voiture  280  -REJET-
[GASOIL] |  164.5 : Voiture  281  -REJET-
[SUPER] | [Minute :  165.0 ]: Voiture  282 arrive, voitures en attente :  3
[SANSPLOMB] |  165.5 : Voiture  283  -REJET-
[GASOIL] |  166.0 : Voiture  284  -REJET-
[SUPER] | [Minute :  166.5 ]: Voiture  285 arrive, voitures en attente :  3
[SUPER] | [Minute :  167.0 ]: Voiture  286 arrive, voitures en attente :  4
[SUP

[GASOIL] | [Minute :  295.5 ] Voiture  497 arrive, voitures en attente :  4
[GASOIL] |  296.0 : Voiture  498  -REJET-
[SUPER] | [Minute :  297.0 ]: Voiture  499 arrive, voitures en attente :  13
[SUPER] | [Minute :  297.5 ]: Voiture  500 arrive, voitures en attente :  11
[GASOIL] |  298.0 : Voiture  501  -REJET-
[GASOIL] | [Minute :  298.5 ] Voiture  502 arrive, voitures en attente :  4
[SUPER] | [Minute :  299.5 ]: Voiture  503 arrive, voitures en attente :  11
[GASOIL] |  300.0 : Voiture  504  -REJET-
[GASOIL] |  301.5 : Voiture  505  -REJET-
[SUPER] | [Minute :  302.0 ]: Voiture  506 arrive, voitures en attente :  11
[GASOIL] |  302.5 : Voiture  507  -REJET-
[SUPER] | [Minute :  303.5 ]: Voiture  508 arrive, voitures en attente :  10
[SUPER] | [Minute :  304.0 ]: Voiture  509 arrive, voitures en attente :  11
[SUPER] | [Minute :  304.5 ]: Voiture  510 arrive, voitures en attente :  11
[SANSPLOMB] | [Minute :  305.0 ]: Voiture  511 arrive, voitures en attente :  1
[GASOIL] |  305.5 :

[SUPER] | [Minute :  443.5 ]: Voiture  741 arrive, voitures en attente :  11
[GASOIL] |  444.5 : Voiture  742  -REJET-
[SUPER] | [Minute :  445.5 ]: Voiture  743 arrive, voitures en attente :  10
[GASOIL] |  446.0 : Voiture  744  -REJET-
[SANSPLOMB] | [Minute :  446.5 ]: Voiture  745 arrive, voitures en attente :  1
[GASOIL] |  447.5 : Voiture  746  -REJET-
[SUPER] | [Minute :  448.0 ]: Voiture  747 arrive, voitures en attente :  10
[GASOIL] |  448.5 : Voiture  748  -REJET-
[SANSPLOMB] | [Minute :  449.0 ]: Voiture  749 arrive, voitures en attente :  1
[GASOIL] |  449.5 : Voiture  750  -REJET-
[SUPER] | [Minute :  450.0 ]: Voiture  751 arrive, voitures en attente :  11
[SUPER] | [Minute :  450.5 ]: Voiture  752 arrive, voitures en attente :  12
[SUPER] | [Minute :  451.0 ]: Voiture  753 arrive, voitures en attente :  13
[SUPER] | [Minute :  452.0 ]: Voiture  754 arrive, voitures en attente :  11
[GASOIL] | [Minute :  452.5 ] Voiture  755 arrive, voitures en attente :  3
[SANSPLOMB] | [

[SUPER] | [Minute :  596.0 ]: Voiture  993 arrive, voitures en attente :  13
[SUPER] | [Minute :  596.5 ]: Voiture  994 arrive, voitures en attente :  14
[SUPER] | [Minute :  597.0 ]: Voiture  995 arrive, voitures en attente :  14
[SUPER] | [Minute :  597.5 ]: Voiture  996 arrive, voitures en attente :  15
[SUPER] | [Minute :  598.5 ]: Voiture  997 arrive, voitures en attente :  16
[GASOIL] | [Minute :  599.0 ] Voiture  998 arrive, voitures en attente :  2
[SUPER] |  599.5 : Voiture  999  -REJET-
[SANSPLOMB] |  600.0 : Voiture  1000  -REJET-
[SUPER] |  600.5 : Voiture  1001  -REJET-
[SANSPLOMB] |  601.0 : Voiture  1002  -REJET-
[GASOIL] | [Minute :  601.5 ] Voiture  1003 arrive, voitures en attente :  2
[SANSPLOMB] |  602.0 : Voiture  1004  -REJET-
[SUPER] | [Minute :  602.5 ]: Voiture  1005 arrive, voitures en attente :  16
[GASOIL] | [Minute :  603.0 ] Voiture  1006 arrive, voitures en attente :  3
[GASOIL] | [Minute :  603.5 ] Voiture  1007 arrive, voitures en attente :  4
[GASOIL] 

[SUPER] | [Minute :  766.5 ]: Voiture  1285 arrive, voitures en attente :  6
[GASOIL] | [Minute :  767.0 ] Voiture  1286 arrive, voitures en attente :  4
[GASOIL] |  767.5 : Voiture  1287  -REJET-
[SANSPLOMB] | [Minute :  768.0 ]: Voiture  1288 arrive, voitures en attente :  4
[SANSPLOMB] |  768.5 : Voiture  1289  -REJET-
[SUPER] | [Minute :  769.0 ]: Voiture  1290 arrive, voitures en attente :  4
[SUPER] | [Minute :  770.0 ]: Voiture  1291 arrive, voitures en attente :  5
[SANSPLOMB] |  771.0 : Voiture  1292  -REJET-
[SANSPLOMB] |  771.5 : Voiture  1293  -REJET-
[SANSPLOMB] |  772.0 : Voiture  1294  -REJET-
[SUPER] | [Minute :  772.5 ]: Voiture  1295 arrive, voitures en attente :  3
[SANSPLOMB] |  773.0 : Voiture  1296  -REJET-
[GASOIL] | [Minute :  773.5 ] Voiture  1297 arrive, voitures en attente :  4
[SUPER] | [Minute :  774.5 ]: Voiture  1298 arrive, voitures en attente :  4
[SUPER] | [Minute :  775.5 ]: Voiture  1299 arrive, voitures en attente :  3
[SUPER] | [Minute :  776.0 ]: 

[SUPER] | [Minute :  936.0 ]: Voiture  1578 arrive, voitures en attente :  14
[SANSPLOMB] | [Minute :  936.5 ]: Voiture  1579 arrive, voitures en attente :  1
[GASOIL] |  937.0 : Voiture  1580  -REJET-
[SUPER] | [Minute :  937.5 ]: Voiture  1581 arrive, voitures en attente :  12
[GASOIL] |  938.0 : Voiture  1582  -REJET-
[SUPER] | [Minute :  938.5 ]: Voiture  1583 arrive, voitures en attente :  12
[GASOIL] |  939.5 : Voiture  1584  -REJET-
[SUPER] | [Minute :  940.0 ]: Voiture  1585 arrive, voitures en attente :  10
[SANSPLOMB] | [Minute :  940.5 ]: Voiture  1586 arrive, voitures en attente :  2
[SUPER] | [Minute :  941.0 ]: Voiture  1587 arrive, voitures en attente :  11
[SANSPLOMB] | [Minute :  941.5 ]: Voiture  1588 arrive, voitures en attente :  3
[SUPER] | [Minute :  942.5 ]: Voiture  1589 arrive, voitures en attente :  11
[GASOIL] | [Minute :  943.0 ] Voiture  1590 arrive, voitures en attente :  4
[SUPER] | [Minute :  944.0 ]: Voiture  1591 arrive, voitures en attente :  10
[SUPE

[SUPER] | [Minute :  1103.0 ]: Voiture  1854 arrive, voitures en attente :  14
[GASOIL] |  1104.0 : Voiture  1855  -REJET-
[SUPER] | [Minute :  1104.5 ]: Voiture  1856 arrive, voitures en attente :  12
[GASOIL] | [Minute :  1105.0 ] Voiture  1857 arrive, voitures en attente :  4
[SUPER] | [Minute :  1105.5 ]: Voiture  1858 arrive, voitures en attente :  10
[GASOIL] |  1106.0 : Voiture  1859  -REJET-
[SUPER] | [Minute :  1106.5 ]: Voiture  1860 arrive, voitures en attente :  11
[GASOIL] |  1107.0 : Voiture  1861  -REJET-
[SUPER] | [Minute :  1107.5 ]: Voiture  1862 arrive, voitures en attente :  12
[SANSPLOMB] | [Minute :  1108.0 ]: Voiture  1863 arrive, voitures en attente :  3
[SUPER] | [Minute :  1108.5 ]: Voiture  1864 arrive, voitures en attente :  12
[SUPER] | [Minute :  1109.0 ]: Voiture  1865 arrive, voitures en attente :  12
[SANSPLOMB] | [Minute :  1109.5 ]: Voiture  1866 arrive, voitures en attente :  3
[GASOIL] |  1110.0 : Voiture  1867  -REJET-
[GASOIL] |  1110.5 : Voiture 

[SANSPLOMB] | [Minute :  1275.5 ]: Voiture  2144 arrive, voitures en attente :  3
[SUPER] | [Minute :  1276.0 ]: Voiture  2145 arrive, voitures en attente :  14
[SUPER] | [Minute :  1277.0 ]: Voiture  2146 arrive, voitures en attente :  15
[SANSPLOMB] | [Minute :  1278.0 ]: Voiture  2147 arrive, voitures en attente :  2
[GASOIL] | [Minute :  1278.5 ] Voiture  2148 arrive, voitures en attente :  4
[SUPER] | [Minute :  1279.0 ]: Voiture  2149 arrive, voitures en attente :  14
[GASOIL] |  1279.5 : Voiture  2150  -REJET-
[SUPER] | [Minute :  1280.5 ]: Voiture  2151 arrive, voitures en attente :  14
[SUPER] | [Minute :  1282.0 ]: Voiture  2152 arrive, voitures en attente :  13
[SUPER] | [Minute :  1282.5 ]: Voiture  2153 arrive, voitures en attente :  13
[SUPER] | [Minute :  1283.0 ]: Voiture  2154 arrive, voitures en attente :  14
[SUPER] | [Minute :  1283.5 ]: Voiture  2155 arrive, voitures en attente :  14
[GASOIL] | [Minute :  1284.0 ] Voiture  2156 arrive, voitures en attente :  4
[SUP

#### Question 2 : Les reservoirs de Gasoil et de sansplom devront etre réaprovisionnés toutes les ~60-65 heures environs, tandis que le reservoir de super deva l'etre toutes les ~30-32 heures.

## Exercice 5 : 

In [ ]:
from numpy.random import uniform
import numpy as np
import simpy

moy_requetes = 200 # une requete toutes les 200ms 
moy_service = 10000 # 10 sec par requete
x=1/moy_requetes
y=1/moy_service

file = []
etat = 0
arrive = []
fin = []
enAttente = []
data = []
arrive = []
fin = []

def arrive_requete():
    return uniform() <=x

def service_requete():
    return uniform() <=y

def traitement_requete(env, num_req, serveur):
    global file, data, fin
    
    file = file+[num_req]
     
    if(env.now % 1000 == 0): 
        data += [len(file)]
        
    print("[HOUR : ","%.2f" % (env.now/(1000*60*60)), " ] : incoming request n°", num_req," requtes en traitement ", len(file))
    
    
    with serveur.put(num_req) as req:
        yield req
        #print(env.now,': debut du traitement du client', num_client)
        temps = 0
        while True : 
            temps = temps+1
            
            if(service_requete()):
                yield env.timeout(temps)
                #print(env.now,': fin du traitement du client ', num_client)
                fin[num_req] = env.now
                temporary = serveur.get()
                file.remove(num_req)
                break
            

def generate_requetes(env,serveur):
    global arrive, fin
    num_req=0
    temps=0
    
    while True:
        temps = temps +1
        if(env.now > 64800000 and env.now < 82800000 and uniform() < 1/10):
            yield env.timeout(temps)
            env.process(traitement_requete(env, num_req, serveur))
            arrive += [env.now]
            fin += [env.now]
            num_req+=1
            temps=0
        elif(arrive_requete()):
            yield env.timeout(temps)
            env.process(traitement_requete(env, num_req, serveur))
            arrive += [env.now]
            fin += [env.now]
            num_req+=1
            temps=0
        
            
            
    
env = simpy.Environment()
antenne = simpy.Store(env)
env.process(generate_requetes(env, antenne))

print('Début de la simulation')
env.run(until=24*60*60*1000)
print('Fin de a simulation')
attente = np.array(fin)-np.array(arrive)
enAttente = np.array(enAttente)

#print('\nAttente moyenne : ', attente.mean(),'Attente max : ', attente.max(), 'Attente min : ' , attente.min()  )


Début de la simulation
[HOUR :  0.00  ] : incoming request n° 0  requtes en traitement  1
[HOUR :  0.00  ] : incoming request n° 1  requtes en traitement  2
[HOUR :  0.00  ] : incoming request n° 2  requtes en traitement  3
[HOUR :  0.00  ] : incoming request n° 3  requtes en traitement  4
[HOUR :  0.00  ] : incoming request n° 4  requtes en traitement  5
[HOUR :  0.00  ] : incoming request n° 5  requtes en traitement  6
[HOUR :  0.00  ] : incoming request n° 6  requtes en traitement  7
[HOUR :  0.00  ] : incoming request n° 7  requtes en traitement  8
[HOUR :  0.00  ] : incoming request n° 8  requtes en traitement  9
[HOUR :  0.00  ] : incoming request n° 9  requtes en traitement  10
[HOUR :  0.00  ] : incoming request n° 10  requtes en traitement  10
[HOUR :  0.00  ] : incoming request n° 11  requtes en traitement  11
[HOUR :  0.00  ] : incoming request n° 12  requtes en traitement  12
[HOUR :  0.00  ] : incoming request n° 13  requtes en traitement  13
[HOUR :  0.00  ] : incoming re

[HOUR :  0.01  ] : incoming request n° 128  requtes en traitement  44
[HOUR :  0.01  ] : incoming request n° 129  requtes en traitement  45
[HOUR :  0.01  ] : incoming request n° 130  requtes en traitement  44
[HOUR :  0.01  ] : incoming request n° 131  requtes en traitement  45
[HOUR :  0.01  ] : incoming request n° 132  requtes en traitement  46
[HOUR :  0.01  ] : incoming request n° 133  requtes en traitement  47
[HOUR :  0.01  ] : incoming request n° 134  requtes en traitement  47
[HOUR :  0.01  ] : incoming request n° 135  requtes en traitement  46
[HOUR :  0.01  ] : incoming request n° 136  requtes en traitement  46
[HOUR :  0.01  ] : incoming request n° 137  requtes en traitement  47
[HOUR :  0.01  ] : incoming request n° 138  requtes en traitement  47
[HOUR :  0.01  ] : incoming request n° 139  requtes en traitement  45
[HOUR :  0.01  ] : incoming request n° 140  requtes en traitement  46
[HOUR :  0.01  ] : incoming request n° 141  requtes en traitement  47
[HOUR :  0.01  ] : i

[HOUR :  0.01  ] : incoming request n° 247  requtes en traitement  48
[HOUR :  0.01  ] : incoming request n° 248  requtes en traitement  49
[HOUR :  0.01  ] : incoming request n° 249  requtes en traitement  49
[HOUR :  0.01  ] : incoming request n° 250  requtes en traitement  50
[HOUR :  0.01  ] : incoming request n° 251  requtes en traitement  50
[HOUR :  0.01  ] : incoming request n° 252  requtes en traitement  51
[HOUR :  0.01  ] : incoming request n° 253  requtes en traitement  52
[HOUR :  0.01  ] : incoming request n° 254  requtes en traitement  51
[HOUR :  0.01  ] : incoming request n° 255  requtes en traitement  52
[HOUR :  0.01  ] : incoming request n° 256  requtes en traitement  50
[HOUR :  0.01  ] : incoming request n° 257  requtes en traitement  50
[HOUR :  0.01  ] : incoming request n° 258  requtes en traitement  50
[HOUR :  0.01  ] : incoming request n° 259  requtes en traitement  50
[HOUR :  0.01  ] : incoming request n° 260  requtes en traitement  51
[HOUR :  0.01  ] : i

[HOUR :  0.02  ] : incoming request n° 371  requtes en traitement  67
[HOUR :  0.02  ] : incoming request n° 372  requtes en traitement  68
[HOUR :  0.02  ] : incoming request n° 373  requtes en traitement  67
[HOUR :  0.02  ] : incoming request n° 374  requtes en traitement  68
[HOUR :  0.02  ] : incoming request n° 375  requtes en traitement  67
[HOUR :  0.02  ] : incoming request n° 376  requtes en traitement  68
[HOUR :  0.02  ] : incoming request n° 377  requtes en traitement  66
[HOUR :  0.02  ] : incoming request n° 378  requtes en traitement  64
[HOUR :  0.02  ] : incoming request n° 379  requtes en traitement  64
[HOUR :  0.02  ] : incoming request n° 380  requtes en traitement  65
[HOUR :  0.02  ] : incoming request n° 381  requtes en traitement  63
[HOUR :  0.02  ] : incoming request n° 382  requtes en traitement  61
[HOUR :  0.02  ] : incoming request n° 383  requtes en traitement  61
[HOUR :  0.02  ] : incoming request n° 384  requtes en traitement  62
[HOUR :  0.02  ] : i

[HOUR :  0.03  ] : incoming request n° 502  requtes en traitement  42
[HOUR :  0.03  ] : incoming request n° 503  requtes en traitement  43
[HOUR :  0.03  ] : incoming request n° 504  requtes en traitement  44
[HOUR :  0.03  ] : incoming request n° 505  requtes en traitement  45
[HOUR :  0.03  ] : incoming request n° 506  requtes en traitement  46
[HOUR :  0.03  ] : incoming request n° 507  requtes en traitement  47
[HOUR :  0.03  ] : incoming request n° 508  requtes en traitement  47
[HOUR :  0.03  ] : incoming request n° 509  requtes en traitement  48
[HOUR :  0.03  ] : incoming request n° 510  requtes en traitement  49
[HOUR :  0.03  ] : incoming request n° 511  requtes en traitement  49
[HOUR :  0.03  ] : incoming request n° 512  requtes en traitement  50
[HOUR :  0.03  ] : incoming request n° 513  requtes en traitement  47
[HOUR :  0.03  ] : incoming request n° 514  requtes en traitement  48
[HOUR :  0.03  ] : incoming request n° 515  requtes en traitement  49
[HOUR :  0.03  ] : i

...

[HOUR :  0.04  ] : incoming request n° 751  requtes en traitement  48
[HOUR :  0.04  ] : incoming request n° 752  requtes en traitement  48
[HOUR :  0.04  ] : incoming request n° 753  requtes en traitement  46
[HOUR :  0.04  ] : incoming request n° 754  requtes en traitement  47
[HOUR :  0.04  ] : incoming request n° 755  requtes en traitement  48
[HOUR :  0.04  ] : incoming request n° 756  requtes en traitement  49
[HOUR :  0.04  ] : incoming request n° 757  requtes en traitement  50
[HOUR :  0.04  ] : incoming request n° 758  requtes en traitement  50
[HOUR :  0.04  ] : incoming request n° 759  requtes en traitement  50
[HOUR :  0.04  ] : incoming request n° 760  requtes en traitement  51
[HOUR :  0.05  ] : incoming request n° 761  requtes en traitement  49
[HOUR :  0.05  ] : incoming request n° 762  requtes en traitement  47
[HOUR :  0.05  ] : incoming request n° 763  requtes en traitement  48
[HOUR :  0.05  ] : incoming request n° 764  requtes en traitement  48
[HOUR :  0.05  ] : i

[HOUR :  0.05  ] : incoming request n° 879  requtes en traitement  44
[HOUR :  0.05  ] : incoming request n° 880  requtes en traitement  45
[HOUR :  0.05  ] : incoming request n° 881  requtes en traitement  43
[HOUR :  0.05  ] : incoming request n° 882  requtes en traitement  42
[HOUR :  0.05  ] : incoming request n° 883  requtes en traitement  43
[HOUR :  0.05  ] : incoming request n° 884  requtes en traitement  44
[HOUR :  0.05  ] : incoming request n° 885  requtes en traitement  45
[HOUR :  0.05  ] : incoming request n° 886  requtes en traitement  41
[HOUR :  0.05  ] : incoming request n° 887  requtes en traitement  41
[HOUR :  0.05  ] : incoming request n° 888  requtes en traitement  42
[HOUR :  0.05  ] : incoming request n° 889  requtes en traitement  43
[HOUR :  0.05  ] : incoming request n° 890  requtes en traitement  44
[HOUR :  0.05  ] : incoming request n° 891  requtes en traitement  45
[HOUR :  0.05  ] : incoming request n° 892  requtes en traitement  46
[HOUR :  0.05  ] : i

[HOUR :  0.06  ] : incoming request n° 1007  requtes en traitement  53
[HOUR :  0.06  ] : incoming request n° 1008  requtes en traitement  53
[HOUR :  0.06  ] : incoming request n° 1009  requtes en traitement  54
[HOUR :  0.06  ] : incoming request n° 1010  requtes en traitement  55
[HOUR :  0.06  ] : incoming request n° 1011  requtes en traitement  54
[HOUR :  0.06  ] : incoming request n° 1012  requtes en traitement  53
[HOUR :  0.06  ] : incoming request n° 1013  requtes en traitement  53
[HOUR :  0.06  ] : incoming request n° 1014  requtes en traitement  54
[HOUR :  0.06  ] : incoming request n° 1015  requtes en traitement  53
[HOUR :  0.06  ] : incoming request n° 1016  requtes en traitement  53
[HOUR :  0.06  ] : incoming request n° 1017  requtes en traitement  54
[HOUR :  0.06  ] : incoming request n° 1018  requtes en traitement  54
[HOUR :  0.06  ] : incoming request n° 1019  requtes en traitement  55
[HOUR :  0.06  ] : incoming request n° 1020  requtes en traitement  56
[HOUR 

[HOUR :  0.07  ] : incoming request n° 1141  requtes en traitement  49
[HOUR :  0.07  ] : incoming request n° 1142  requtes en traitement  50
[HOUR :  0.07  ] : incoming request n° 1143  requtes en traitement  49
[HOUR :  0.07  ] : incoming request n° 1144  requtes en traitement  48
[HOUR :  0.07  ] : incoming request n° 1145  requtes en traitement  49
[HOUR :  0.07  ] : incoming request n° 1146  requtes en traitement  50
[HOUR :  0.07  ] : incoming request n° 1147  requtes en traitement  51
[HOUR :  0.07  ] : incoming request n° 1148  requtes en traitement  51
[HOUR :  0.07  ] : incoming request n° 1149  requtes en traitement  51
[HOUR :  0.07  ] : incoming request n° 1150  requtes en traitement  50
[HOUR :  0.07  ] : incoming request n° 1151  requtes en traitement  49
[HOUR :  0.07  ] : incoming request n° 1152  requtes en traitement  49
[HOUR :  0.07  ] : incoming request n° 1153  requtes en traitement  49
[HOUR :  0.07  ] : incoming request n° 1154  requtes en traitement  49
[HOUR 

[HOUR :  0.07  ] : incoming request n° 1272  requtes en traitement  64
[HOUR :  0.07  ] : incoming request n° 1273  requtes en traitement  64
[HOUR :  0.07  ] : incoming request n° 1274  requtes en traitement  64
[HOUR :  0.07  ] : incoming request n° 1275  requtes en traitement  64
[HOUR :  0.07  ] : incoming request n° 1276  requtes en traitement  62
[HOUR :  0.07  ] : incoming request n° 1277  requtes en traitement  63
[HOUR :  0.07  ] : incoming request n° 1278  requtes en traitement  64
[HOUR :  0.07  ] : incoming request n° 1279  requtes en traitement  65
[HOUR :  0.07  ] : incoming request n° 1280  requtes en traitement  66
[HOUR :  0.07  ] : incoming request n° 1281  requtes en traitement  67
[HOUR :  0.07  ] : incoming request n° 1282  requtes en traitement  66
[HOUR :  0.07  ] : incoming request n° 1283  requtes en traitement  67
[HOUR :  0.07  ] : incoming request n° 1284  requtes en traitement  68
[HOUR :  0.07  ] : incoming request n° 1285  requtes en traitement  69
[HOUR 

In [3]:
debit = [500/x for x in data]


113150.08974801519